In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime

import glob
import dask
import warnings
import cartopy
import pickle

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import sys
sys.path.insert(0, "/user/work/eh19374/LPDM-emulation-trees_TESTING/")
from trees_emulator.load_data import *
from trees_emulator.training import *
from trees_emulator.predicting import *
from trees_emulator.marco_functions import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
# run this so loaded functions/packages get automatically updated if you edit
%load_ext autoreload
%autoreload 2

### Loading Datasets

In [3]:
# load the data for a particular site
domains = {"MHD":"EUROPE", "THD":"USA", "TAC":"EUROPE", "RGL":"EUROPE", "HFD":"EUROPE", "BSD":"EUROPE", "GSN":"EASTASIA"}
heights = {"MHD":"10magl", "THD":"10magl", "TAC":"185magl", "RGL":"90magl", "HFD":"100magl", "BSD":"250magl", "GSN":"10magl"} # default heights

## for UK/Ireland sites (MHD, TAC, RGL, BSD, HFD)
met_datadir = "/group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*"
extramet_datadir = "/group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*"


In [4]:
site = 'MHD'
fp_datadir = f"/group/chemistry/acrg/LPDM/fp_NAME/EUROPE/{site}-{heights[site]}_UKV_EUROPE_*"
MHD_2016 = LoadData(year="2016", site=site, size=10, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
MHD_2016_coarse2 = LoadData(year="2016", site=site, size=9, coarsen_factor=2, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
MHD_2016_coarse3 = LoadData(year="2016", site=site, size=12, coarsen_factor=3, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
MHD_2016_coarse4 = LoadData(year="2016", site=site, size=17, coarsen_factor=4, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
MHD_2016_coarse5 = LoadData(year="2016", site=site, size=22, coarsen_factor=5, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)

Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/MHD-10magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/MHD-10magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/MHD-10magl_UKV_EUROPE_*201

In [5]:
site = 'TAC'
fp_datadir = f"/group/chemistry/acrg/LPDM/fp_NAME/EUROPE/{site}-{heights[site]}_UKV_EUROPE_*"
TAC_2016 = LoadData(year="2016", site=site, size=10, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
TAC_2016_coarse2 = LoadData(year="2016", site=site, size=9, coarsen_factor=2, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
TAC_2016_coarse3 = LoadData(year="2016", site=site, size=12, coarsen_factor=3, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
TAC_2016_coarse4 = LoadData(year="2016", site=site, size=17, coarsen_factor=4, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
TAC_2016_coarse5 = LoadData(year="2016", site=site, size=22, coarsen_factor=5, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)

Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/TAC-185magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/TAC-185magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/TAC-185magl_UKV_EUROPE_*

In [6]:
site = 'RGL'
fp_datadir = f"/group/chemistry/acrg/LPDM/fp_NAME/EUROPE/{site}-{heights[site]}_UKV_EUROPE_*"
RGL_2016 = LoadData(year="2016", site=site, size=10, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
RGL_2016_coarse2 = LoadData(year="2016", site=site, size=9, coarsen_factor=2, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
RGL_2016_coarse3 = LoadData(year="2016", site=site, size=12, coarsen_factor=3, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
RGL_2016_coarse4 = LoadData(year="2016", site=site, size=17, coarsen_factor=4, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
RGL_2016_coarse5 = LoadData(year="2016", site=site, size=22, coarsen_factor=5, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)

Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/RGL-90magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/RGL-90magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/RGL-90magl_UKV_EUROPE_*201

In [7]:
site = 'BSD'
fp_datadir = f"/group/chemistry/acrg/LPDM/fp_NAME/EUROPE/{site}-{heights[site]}_UKV_EUROPE_*"
BSD_2016 = LoadData(year="2016", site=site, size=10, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
BSD_2016_coarse2 = LoadData(year="2016", site=site, size=9, coarsen_factor=2, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
BSD_2016_coarse3 = LoadData(year="2016", site=site, size=12, coarsen_factor=3, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
BSD_2016_coarse4 = LoadData(year="2016", site=site, size=17, coarsen_factor=4, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
BSD_2016_coarse5 = LoadData(year="2016", site=site, size=22, coarsen_factor=5, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)

Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/BSD-250magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/BSD-250magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/BSD-250magl_UKV_EUROPE_*

In [8]:
site = 'HFD'
fp_datadir = f"/group/chemistry/acrg/LPDM/fp_NAME/EUROPE/{site}-{heights[site]}_UKV_EUROPE_*"
HFD_2016 = LoadData(year="2016", site=site, size=10, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
HFD_2016_coarse2 = LoadData(year="2016", site=site, size=9, coarsen_factor=2, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
HFD_2016_coarse3 = LoadData(year="2016", site=site, size=12, coarsen_factor=3, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
HFD_2016_coarse4 = LoadData(year="2016", site=site, size=17, coarsen_factor=4, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)
HFD_2016_coarse5 = LoadData(year="2016", site=site, size=22, coarsen_factor=5, verbose=True, met_datadir=met_datadir, fp_datadir=fp_datadir, extramet_datadir=extramet_datadir)

Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/HFD-100magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/HFD-100magl_UKV_EUROPE_*2016*.nc
Cutting data to size
Loading extra meteorology from /group/chemistry/acrg/met_archive/NAME/full_extra_vars/EUROPE*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded
Loading Meteorology data from /group/chemistry/acrg/met_archive/NAME/EUROPE_met/EUROPE_Met_10magl_*2016*.nc
Loading footprint data from /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/HFD-100magl_UKV_EUROPE_*

### TRAINING

In [9]:
# MHD
MHD_2 = import_coarsened_and_train('MHD_1415_coarsened2', MHD_2016_coarse2, [6], 14)
MHD_3 = import_coarsened_and_train('MHD_1415_coarsened3', MHD_2016_coarse3, [6], 18)
MHD_4 = import_coarsened_and_train('MHD_1415_coarsened4', MHD_2016_coarse4, [6], 22)
MHD_5 = import_coarsened_and_train('MHD_1415_coarsened5', MHD_2016_coarse5, [6], 28)


Trained model info: {'site': 'MHD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 9}
Trained model info: {'site': 'MHD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 12}
Trained model info: {'site': 'MHD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 17}
Trained model info: {'site': 'MHD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 22}


In [10]:
# TAC
TAC_2 = import_coarsened_and_train('TAC_1415_coarsened2', TAC_2016_coarse2, [6], 14)
TAC_3 = import_coarsened_and_train('TAC_1415_coarsened3', TAC_2016_coarse3, [6], 18)
TAC_4 = import_coarsened_and_train('TAC_1415_coarsened4', TAC_2016_coarse4, [6], 22)
TAC_5 = import_coarsened_and_train('TAC_1415_coarsened5', TAC_2016_coarse5, [6], 28)

Trained model info: {'site': 'TAC', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 9}
Trained model info: {'site': 'TAC', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 12}
Trained model info: {'site': 'TAC', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 17}
Trained model info: {'site': 'TAC', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 22}


In [11]:
# RGL
RGL_2 = import_coarsened_and_train('RGL_1415_coarsened2', RGL_2016_coarse2, [6], 14)
RGL_3 = import_coarsened_and_train('RGL_1415_coarsened3', RGL_2016_coarse3, [6], 18)
RGL_4 = import_coarsened_and_train('RGL_1415_coarsened4', RGL_2016_coarse4, [6], 22)
RGL_5 = import_coarsened_and_train('RGL_1415_coarsened5', RGL_2016_coarse5, [6], 28)

Trained model info: {'site': 'RGL', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 9}
Trained model info: {'site': 'RGL', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 12}
Trained model info: {'site': 'RGL', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 17}
Trained model info: {'site': 'RGL', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 22}


In [12]:
# BSD
BSD_2 = import_coarsened_and_train('BSD_1415_coarsened2', BSD_2016_coarse2, [6], 14)
BSD_3 = import_coarsened_and_train('BSD_1415_coarsened3', BSD_2016_coarse3, [6], 18)
BSD_4 = import_coarsened_and_train('BSD_1415_coarsened4', BSD_2016_coarse4, [6], 22)
BSD_5 = import_coarsened_and_train('BSD_1415_coarsened5', BSD_2016_coarse5, [6], 28)

Trained model info: {'site': 'BSD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 9}
Trained model info: {'site': 'BSD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 12}
Trained model info: {'site': 'BSD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 17}
Trained model info: {'site': 'BSD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 22}


In [13]:
# HFD
HFD_2 = import_coarsened_and_train('HFD_1415_coarsened2', HFD_2016_coarse2, [6], 14)
HFD_3 = import_coarsened_and_train('HFD_1415_coarsened3', HFD_2016_coarse3, [6], 18)
HFD_4 = import_coarsened_and_train('HFD_1415_coarsened4', HFD_2016_coarse4, [6], 22)
HFD_5 = import_coarsened_and_train('HFD_1415_coarsened5', HFD_2016_coarse5, [6], 28)

Trained model info: {'site': 'HFD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 9}
Trained model info: {'site': 'HFD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 12}
Trained model info: {'site': 'HFD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 17}
Trained model info: {'site': 'HFD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 22}


In [16]:
def Uncoarsen(dataset, dataset_coarsened, dataOG, coarsen_factor, show_plot=False, date=0):
    data = dataset.predictions
    data_fp_coarse = dataset_coarsened.fp_data

    data = data.reshape(len(data), dataset.size, dataset.size)
    data_fp_coarse = data_fp_coarse.reshape(len(data_fp_coarse), dataset.size, dataset.size)
    data_fp_coarse = data_fp_coarse[dataset.jump:-3]
    
    array = xr.DataArray(data, coords=[('time', dataset.data.met.time.values[dataset.jump:-3]),
                                        ('lat', dataset.data.fp_lats),
                                        ('lon', dataset.data.fp_lons)])
    
    fp = xr.DataArray(data_fp_coarse, coords=[('time', dataset.data.met.time.values[dataset.jump:-3]),
                                            ('lat', dataset.data.fp_lats),
                                            ('lon', dataset.data.fp_lons)])
    
    new_lats = dataOG.fp_data_full.lat
    new_lons = dataOG.fp_data_full.lon
    
    array = array.interp(lat=new_lats, lon=new_lons, method='nearest')
    array = xr.where((array > 0) & ~np.isnan(array), array/coarsen_factor, array)

    fp = fp.interp(lat=new_lats, lon=new_lons, method='nearest')
    fp = xr.where((fp > 0) & ~np.isnan(fp), fp/coarsen_factor, fp)

    if show_plot==True:
        plot_xarray(array, date, dataset.data.site, coarsen_factor)

    return array, fp


In [17]:
# MHD
MHD_2_uncoarsened_predictions, MHD_2_uncoarsened_fp_data = Uncoarsen(MHD_2, MHD_2016_coarse2, MHD_2016, 2)
MHD_3_uncoarsened_predictions, MHD_3_uncoarsened_fp_data = Uncoarsen(MHD_3, MHD_2016_coarse3, MHD_2016, 3)
MHD_4_uncoarsened_predictions, MHD_4_uncoarsened_fp_data = Uncoarsen(MHD_4, MHD_2016_coarse4, MHD_2016, 4)
MHD_5_uncoarsened_predictions, MHD_5_uncoarsened_fp_data = Uncoarsen(MHD_5, MHD_2016_coarse5, MHD_2016, 5)

In [18]:
# TAC
TAC_2_uncoarsened_predictions, TAC_2_uncoarsened_fp_data = Uncoarsen(TAC_2, TAC_2016_coarse2, TAC_2016, 2)
TAC_3_uncoarsened_predictions, TAC_3_uncoarsened_fp_data = Uncoarsen(TAC_3, TAC_2016_coarse3, TAC_2016, 3)
TAC_4_uncoarsened_predictions, TAC_4_uncoarsened_fp_data = Uncoarsen(TAC_4, TAC_2016_coarse4, TAC_2016, 4)
TAC_5_uncoarsened_predictions, TAC_5_uncoarsened_fp_data = Uncoarsen(TAC_5, TAC_2016_coarse5, TAC_2016, 5)

In [19]:
# RGL
RGL_2_uncoarsened_predictions, RGL_2_uncoarsened_fp_data = Uncoarsen(RGL_2, RGL_2016_coarse2, RGL_2016, 2)
RGL_3_uncoarsened_predictions, RGL_3_uncoarsened_fp_data = Uncoarsen(RGL_3, RGL_2016_coarse3, RGL_2016, 3)
RGL_4_uncoarsened_predictions, RGL_4_uncoarsened_fp_data = Uncoarsen(RGL_4, RGL_2016_coarse4, RGL_2016, 4)
RGL_5_uncoarsened_predictions, RGL_5_uncoarsened_fp_data = Uncoarsen(RGL_5, RGL_2016_coarse5, RGL_2016, 5)

In [20]:
# BSD
BSD_2_uncoarsened_predictions, BSD_2_uncoarsened_fp_data = Uncoarsen(BSD_2, BSD_2016_coarse2, BSD_2016, 2)
BSD_3_uncoarsened_predictions, BSD_3_uncoarsened_fp_data = Uncoarsen(BSD_3, BSD_2016_coarse3, BSD_2016, 3)
BSD_4_uncoarsened_predictions, BSD_4_uncoarsened_fp_data = Uncoarsen(BSD_4, BSD_2016_coarse4, BSD_2016, 4)
BSD_5_uncoarsened_predictions, BSD_5_uncoarsened_fp_data = Uncoarsen(BSD_5, BSD_2016_coarse5, BSD_2016, 5)

: 

In [ ]:
# HFD
HFD_2_uncoarsened_predictions, HFD_2_uncoarsened_fp_data = Uncoarsen(HFD_2, HFD_2016_coarse2, HFD_2016, 2)
HFD_3_uncoarsened_predictions, HFD_3_uncoarsened_fp_data = Uncoarsen(HFD_3, HFD_2016_coarse3, HFD_2016, 3)
HFD_4_uncoarsened_predictions, HFD_4_uncoarsened_fp_data = Uncoarsen(HFD_4, HFD_2016_coarse4, HFD_2016, 4)
HFD_5_uncoarsened_predictions, HFD_5_uncoarsened_fp_data = Uncoarsen(HFD_5, HFD_2016_coarse5, HFD_2016, 5)

In [ ]:
def combine_all(list_of_xarrays, original_data, return_all=False):
    # combine uncoarsened datasets
    combined = list_of_xarrays[0]
    for x in range(len(list_of_xarrays)-1):
        combined = combined.combine_first(list_of_xarrays[x+1])
    
    combined = xr.where((combined < 0.0005) & ~np.isnan(combined), 0, combined)

    # place OG at centre
    OG_data = original_data.fp_data.reshape(-1, 10, 10)
    OG_data = OG_data[6:-3]

    # Find indices in combined that correspond to the indices in OG_data
    og_lat_indices = np.searchsorted(combined.lat, original_data.fp_lats)
    og_lon_indices = np.searchsorted(combined.lon, original_data.fp_lons)

    # Replace values using the indices
    combined[:, og_lat_indices, og_lon_indices] = OG_data

    # Create copy of OG xarray to use to fill surrounding values
    fp_data = np.array(original_data.fp_data_full.fp)
    fp_data = np.transpose(fp_data, (2,0,1))[6:-3]
    OG_copy = xr.DataArray(fp_data, coords=[('time', np.array(original_data.fp_data_full.time[6:-3])),
                                            ('lat', np.array(original_data.fp_data_full.lat)),
                                            ('lon', np.array(original_data.fp_data_full.lon))])

    final = xr.where(~np.isnan(combined), combined, OG_copy)

    if return_all == True: return final, combined, OG_copy
    else: return final
    

In [ ]:
MHD_uncoarsened = [MHD_2_uncoarsened_predictions, 
                   MHD_3_uncoarsened_predictions, 
                   MHD_4_uncoarsened_predictions, 
                   MHD_5_uncoarsened_predictions]

MHD_final, MHD_combined, MHD_OG = combine_all(MHD_uncoarsened, MHD_2016, True)

In [ ]:
# plot_xarray(MHD_final, 950, 'MHD', 5)
# plt.savefig('plots/MHD_final_mean_threshold_0005.png')

In [ ]:
TAC_uncoarsened = [TAC_2_uncoarsened_predictions, 
                   TAC_3_uncoarsened_predictions, 
                   TAC_4_uncoarsened_predictions, 
                   TAC_5_uncoarsened_predictions]

TAC_final, TAC_combined, TAC_OG = combine_all(TAC_uncoarsened, TAC_2016, True)

In [ ]:
# plot_xarray(TAC_final, 950, 'TAC', 5)
# plt.savefig('plots/TAC_final_mean_threshold_005.png')

In [ ]:
RGL_uncoarsened = [RGL_2_uncoarsened_predictions, 
                   RGL_3_uncoarsened_predictions, 
                   RGL_4_uncoarsened_predictions, 
                   RGL_5_uncoarsened_predictions]

RGL_final, RGL_combined, RGL_OG = combine_all(RGL_uncoarsened, RGL_2016, True)

In [ ]:
# plot_xarray(RGL_final, 950, 'RGL', 5)
# plt.savefig('plots/RGL_final_mean_threshold_none.png')

In [ ]:
BSD_uncoarsened = [BSD_2_uncoarsened_predictions, 
                   BSD_3_uncoarsened_predictions, 
                   BSD_4_uncoarsened_predictions, 
                   BSD_5_uncoarsened_predictions]

BSD_final, BSD_combined, BSD_OG = combine_all(BSD_uncoarsened, BSD_2016, True)

In [ ]:
# plot_xarray(BSD_final, 950, 'BSD', 5)
# plt.savefig('plots/BSD_final_mean_threshold_005.png')

In [ ]:
HFD_uncoarsened = [HFD_2_uncoarsened_predictions, 
                   HFD_3_uncoarsened_predictions, 
                   HFD_4_uncoarsened_predictions, 
                   HFD_5_uncoarsened_predictions]

HFD_final, HFD_combined, HFD_OG = combine_all(HFD_uncoarsened, HFD_2016, True)

In [ ]:
# plot_xarray(HFD_final, 950, 'HFD', 5)
# plt.savefig('plots/HFD_final_mean_threshold_005.png')

In [ ]:
def trim_coarsened_and_OG(coarsened_combined, OG_data):
    combined = coarsened_combined.dropna(dim='lat', how='all')
    combined = combined.dropna(dim='lon', how='all')
    combined_data = combined.data

    lat_mask = OG_data.lat.isin(combined.lat.values)
    lon_mask = OG_data.lon.isin(combined.lon.values)

    filtered_OG = OG_data.where(lat_mask, drop=True).where(lon_mask, drop=True)
    filtered_OG_data = filtered_OG.data

    return combined_data, filtered_OG_data

In [ ]:
def binarize_array(array, threshold):
    # Create a new array of the same shape as the input array
    binarized_array = np.zeros_like(array, dtype=np.int)
    
    # Set values greater than or equal to the threshold to 1
    binarized_array[array >= threshold] = 1
    
    return binarized_array

In [ ]:
def plot_IoU_dice(iou_array, dice_array, site, coarsened_OG=False):
    fig, ax = plt.subplots(1,2, figsize=(10,5))
    ax[0].hist(iou_array, bins=50)
    ax[0].set_title(f'{site} IoU\n(mean = {iou_array.mean():.2f})')

    ax[1].hist(dice_array, bins=50)
    ax[1].set_title(f'{site} Dice Similarity\n(mean = {dice_array.mean():.2f})')

    if coarsened_OG == True: plt.suptitle('{site} predictions vs coarsened truths')
    else: plt.suptitle(f'{site}\npredictions vs original truths')
    plt.show()

In [ ]:
MHD_5x_data, MHD_OG_data = trim_coarsened_and_OG(MHD_final, MHD_OG)

In [ ]:
threshold = 0.0005
MHD_binary = binarize_array(MHD_5x_data, threshold).reshape(11130, 8775)
MHD_binary_OG = binarize_array(MHD_OG_data, threshold).reshape(11130, 8775)

In [ ]:
MHD_IoU = intersection_over_union(MHD_binary, MHD_binary_OG, zero=0)
MHD_dice = dice_similarity(MHD_binary, MHD_binary_OG, zero=0)

In [ ]:
plot_IoU_dice(MHD_IoU, MHD_dice, 'MHD')